# Palabras clave:

Bayes, Naive Bayes Multinomial y Gaussiano, Probabilidad, Probabilidad condicional, Likelihood, Verosimilitud, Clasificación, Tokenización, Stemming, Lemmatización, Distribución, Stop Words, NLTK, Count Vectorizer, Vocabulario, sklearn, dataframe, pandas, sparse matrix, NLP.

In [0]:
# Estos dos comandos evitan que haya que hacer reload cada vez que se modifica un paquete
%load_ext autoreload
%autoreload 2

# Teorema de Bayes

Analicemos el siguiente diagrama:

![Diagrama de Venn](venn.png "Probabilidades de A y B")

Recordemos:

$P(A)$: Probabilidad de que ocurra un evento A.  
$P(B)$: Probabilidad de que ocurra un evento B.  
$P(A\cap B)$: Probabilidad de que ocurran los eventos A y B en simultáneo. Si son independientes: $P(A\cap B)=P(A).P(B)$.  

$P(A|B)$: Probabilidad de que haya ocurrido el evento A,si sabemos que ocurrió el evento B. Si son independientes: $P(A|B)=\frac{P(A\cap B)}{P(B)}$.

También podemos analizar qué pasa para el caso de $P(B|A)$, es decir, la probabilidad de que haya ocurrido el evento B, si sabemos que ocurrió el evento A. $P(B|A)=\frac{P(B\cap A)}{P(A)}$

Igualando $P(B\cap A)$ con $P(A\cap B)$ nos queda:

$P(A|B).P(B)=P(B|A).P(A)$

Despejando:

$$P(B|A)=\frac{P(A|B).P(B)}{P(A)}$$

A la ecuación anterior se la conoce como corolario del Teorema de Bayes. Es de particular interés en el ámbito de Machine Learning ya que a partir de un conjunto de observaciones nos permite estimar qué modelo dió origen a dichas observaciones.

Supongamos que tenemos causas $B_i$ y  efectos $A_j$. Las $B_i$ causas, pueden generar $A_j$ efectos.

Los efectos $A_j$ son exhaustivos es decir: $\sum_kP(A_j)=1$.
Las causas $B_i$ también son exhaustivas.

$P(B_i)$ es la probabilidad de que se dé la causa $B_i$.
$P(A_j)$ es la probabilidad de haber medido el efecto $A_j$.

Un ejemplo:

Tengo una bolsa con cinco monedas del tipo 1, cuya $p_1=0.3$, tres monedas del tipo 2, con $p_2=0.5$ y dos monedas del tipo 3, con  $p_3=0.8$.

Es decir, la distribución de probabilidad de medir el efecto A (que salga H o T) estará condicionada por la causa B (usar una moneda de tipo 1, 2 o 3).

$P(B_i)$ Es la probabilidad de sacar de la bolsa una moneda tipo $i$. Por lo tanto:

|   |  tipo 1 <br>(5 monedas)<br>(50%) |  tipo 2 <br>(3 monedas)<br>(30%) |  tipo 3 <br>(2 monedas)<br>(20%) |
|---|---|---|---|
| H (cara)  |  p1 = 0.3 | p2 = 0.5  | p3 = 0.8  |
| T (seca)  |  1 - p1 = 0.7 |  1 - p2 = 0.5 | 1 - p3 = 0.2  |

Supongamos ahora que las monedas son indistinguibles entre sí y solo tengo el dato de las $P(B_i)$. Una forma de sacar una moneda al azar de la bolsa y determinar a qué tipo pertenece, es tirarla n veces y hacerme la siguiente pregunta:

¿Qué probabilidad hay de haber sacado una moneda del tipo 1 y haber obtenido este resultado? ¿Y de haber sacado una moneda del tipo 2 y haber obtenido este resultado? ¿Y tipo 3?

Es decir nos interesa estimar las probabilidades $P(B_1|A_j)$, $P(B_2|A_j)$ y $P(B_3|A_j)$.

Nuevamente: $P(B_1|A_j)$ puede leerse como "La probabilidad de haber sacado una moneda del tipo 1 si el resultado de arrojarla n veces fue $A_j$. Si calculamos todos los $P(B_i|A_j)$ podemos compararlos entre sí y decir que la causa $B_i$ fue aquella que tiene la probabilidad $P(B_i|A_j)$ mas alta.

Si aplicamos el Teorema de Bayes:

$$P(B_i|A_j)=\frac{P(A_j|B_i).P(B_i)}{P(A_j)}$$

Si sólo nos interesa comparar las $P(B_i|A_j)$ para saber cuál es la causa mas probable, podemos evitar calcular $P(A_j)$ ya que es el mismo para todas las $P(B_i|A_j)$ y solo comparar los numeradores. Podemos definir:

$$\mathcal{L}_i=P(A_j|B_i).P(B_i)$$

A $\mathcal{L}_i$ se lo denomina "medida de verosimilitud de la causa o modelo i". En inglés se lo denomina Likelihood.

Elegir qué modelo es mas probable que haya generado ese efecto (u observación) es equivalente a elegir al modelo cuya medida de verosimilitud sea mayor.

Volviendo a la $P(A_j)$ que dejamos de lado, la misma puede ser calculada como:

$$P(A_j)=\sum_k{P(A_j|B_k).P(B_k)}$$

Por lo tanto, las probabilidades $P(B_i|A_j)$ equivalen al Likelihood normalizado. Es decir:

$$P(B_i|A_j)=\frac{\mathcal{L}_i}{\sum_k{\mathcal{L}_k}}=\frac{P(A_j|B_i).P(B_i)}{\sum_k{P(A_j|B_k).P(B_k)}}$$


Apliquemos ahora estos conceptos a resolver el siguiente problema:

Dadas las moneda en la bolsa descriptas anteriormente, se realiza el siguiente experimento:

Se saca una moneda al azar, se tira 3 veces la moneda y se obtiene el siguiente resultado:

H-H-T

- Estimar el tipo de moneda
- Calcular la probabilidad de que la estimación sea errónea
- Calcular la probabilidad de que la moneda sea de tipo 3

Repetir el problema si lo que se obtuvo fue H-H-T-T-H-T-H

In [0]:


#Calculo el likelihood del modelo 1:
p1=0.3 #probabilidad de sacar cara para la moneda del tipo 1
p2=0.5
p3=0.8
PB1=0.5 #Probabilidad de sacar una moneda tipo 1 de la bolsa
PB2=0.3
PB3=0.2
L1=p1*p1*(1-p1)*PB1
L2=p2*p2*(1-p2)*PB2
L3=p3*p3*(1-p3)*PB3

print("La medida de verosimilitud del modelo 1 para la observación H-H-T es:{}".format(L1))
print("La medida de verosimilitud del modelo 2 para la observación H-H-T es:{}".format(L2))
print("La medida de verosimilitud del modelo 3 para la observación H-H-T es:{}".format(L3))

La medida de verosimilitud del modelo 1 para la observación H-H-T es:0.0315
La medida de verosimilitud del modelo 2 para la observación H-H-T es:0.0375
La medida de verosimilitud del modelo 3 para la observación H-H-T es:0.0256


Por lo tanto el modelo mas probable es el 2. Para saber cual es la probabilidad de que la estimación sea errónea tenemos que calcular las probabilidades condicionales. Lo podemos hacer normalizando con respecto a la suma de los Likelihoods.

In [0]:
print("La probabilidad de que la estimación sea correcta es:{}".format(L2/(L1+L2+L3)))
print("La probabilidad de que la estimación sea errónea es:{}".format(1-L2/(L1+L2+L3)))
print("La probabilidad de que la moneda sea del tipo 3 es:{}".format(L3/(L1+L2+L3)))

La probabilidad de que la estimación sea correcta es:0.3964059196617336
La probabilidad de que la estimación sea errónea es:0.6035940803382664
La probabilidad de que la moneda sea del tipo 3 es:0.27061310782241016


Por lo tanto si bien la estimación es que la moneda es del tipo 2, lo mas probable es que estemos equivocados. Para H-H-T-T-H-T-H:

In [0]:
L1=p1**4*(1-p1)**3*PB1
L2=p2**4*(1-p2)**3*PB2
L3=p3**4*(1-p3)**3*PB3

print("La medida de verosimilitud del modelo 1 para la observación H-H-T es:{}".format(L1))
print("La medida de verosimilitud del modelo 2 para la observación H-H-T es:{}".format(L2))
print("La medida de verosimilitud del modelo 3 para la observación H-H-T es:{}".format(L3))

La medida de verosimilitud del modelo 1 para la observación H-H-T es:0.0013891499999999996
La medida de verosimilitud del modelo 2 para la observación H-H-T es:0.00234375
La medida de verosimilitud del modelo 3 para la observación H-H-T es:0.0006553599999999998


In [0]:
print("La probabilidad de que la estimación sea correcta es:{}".format(L2/(L1+L2+L3)))
print("La probabilidad de que la estimación sea errónea es:{}".format(1-L2/(L1+L2+L3)))
print("La probabilidad de que la moneda sea del tipo 3 es:{}".format(L3/(L1+L2+L3)))

La probabilidad de que la estimación sea correcta es:0.5340955184970808
La probabilidad de que la estimación sea errónea es:0.4659044815029192
La probabilidad de que la moneda sea del tipo 3 es:0.1493439313076253


A medida que aumenta la cantidad de muestras la estimación se va haciendo cada vez mas precisa, pero los valores de probabilidad son cada vez mas chicos, lo cual puede traer problemas para observaciones muy grandes por ejemplo, clasificación de artículos periodísticos o detección de SPAM, ya que estaría compuesto una gran cantidad de multiplicaciones de números mucho mas chicos que 1. Por ello es que se suele utilizar el Log-Likelihood ($\mathcal{logL}$).

El logaritmo es monótono creciente, por lo que la relación entre los Likelihoods no se ve alterada a la hora de compararlos.  
Las multiplicaciones se convierten en suma, por lo que es mucho menos probable que estemos trabajando en los límites de la precisión del procesador.

Para el caso del H-H-T-T-H-T-H los $\mathcal{logL}_i$ quedan:

In [0]:
import math
logL1=4*math.log(p1)+3*math.log(1-p1)+math.log(PB1)
logL2=4*math.log(p2)+3*math.log(1-p2)+math.log(PB2)
logL3=4*math.log(p3)+3*math.log(1-p3)+math.log(PB3)

print("El Log-Likelihood del modelo 1 para la observación H-H-T-T-H-T-H es:{}".format(logL1))
print("El Log-Likelihood del modelo 2 para la observación H-H-T-T-H-T-H es:{}".format(logL2))
print("El Log-Likelihood del modelo 3 para la observación H-H-T-T-H-T-H es:{}".format(logL3))


El Log-Likelihood del modelo 1 para la observación H-H-T-T-H-T-H es:-6.579063229679887
El Log-Likelihood del modelo 2 para la observación H-H-T-T-H-T-H es:-6.056003068245553
El Log-Likelihood del modelo 3 para la observación H-H-T-T-H-T-H es:-7.330325854993241
